In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df1 = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
len(df.select_dtypes(include='object').columns)

In [ ]:
len(df.select_dtypes(exclude='object').columns)

In [ ]:
df.describe()

In [ ]:
j = 1
a = (len(df.select_dtypes(exclude='object').columns)//3)+1
plt.figure(figsize=(20,a*5))
for i in df.select_dtypes(exclude='object'):
    plt.subplot(a,3,j)
    sns.distplot(df[i])
    plt.axvline(df[i].min(),c='b',label='min')
    plt.axvline(df[i].quantile(.25),c='orange',label='25%')
    plt.axvline(df[i].median(),c='y',label='median')
    plt.axvline(df[i].mean(),c='g',label='mean')
    plt.axvline(df[i].quantile(.75),c='brown',label='75%')
    plt.axvline(df[i].max(),c='r',label='max')
    j=j+1
    
plt.legend()

In [ ]:
a = (len(df.select_dtypes(include='object').columns)//3)+1
j=1
plt.figure(figsize=(20,a*5))
for i in df.select_dtypes(include='object'):
    plt.subplot(a,3,j)
    sns.boxplot(y=df['SalePrice'],x=df[i])
    j=j+1

In [ ]:
a = (len(df.select_dtypes(include='object').columns)//3)+1
j=1
plt.figure(figsize=(20,a*5))
for i in df.select_dtypes(include='object'):
    plt.subplot(a,3,j)
    sns.violinplot(y=df['SalePrice'],x=df[i])
    j=j+1

In [ ]:
plt.figure(figsize=(25,20))
sns.heatmap(df.corr(),annot=True,fmt='.1f')

In [ ]:
df.head()

In [ ]:
df.shape,df1.shape

In [ ]:
df.dtypes

In [ ]:
na = df.isna().sum()/len(df)

In [ ]:
na[na>.50]

In [ ]:
df = df.drop(columns=['PoolQC','Id','Alley','Fence','MiscFeature'])
df1 = df1.drop(columns=['PoolQC','Id','Alley','Fence','MiscFeature'])

In [ ]:
a = df.corr()['SalePrice']
cols = list(a[a>0.6].index)
print(cols)

In [ ]:
print(a[a>0.6])

In [ ]:
sns.pairplot(data=df[cols+['ExterQual']],hue='ExterQual')

In [ ]:
plt.figure(figsize=(20,10))
sns.jointplot(x=df['GrLivArea'],y=df['SalePrice'],hue=df['KitchenQual'])

In [ ]:
obj = list(df.select_dtypes(include='object').columns)
len(obj)

In [ ]:
for i in obj:
    df[i] = df[i].fillna('aaaaaaaa')
    df[i] = df[i].astype(str)
for i in obj:
    df1[i] = df1[i].fillna('aaaaaaaa')
    df1[i] = df1[i].astype(str)

In [ ]:
import scipy.stats as stats

col = []
pval = []

In [ ]:
for i in obj:
        
        a = [df[df[i]==j]['SalePrice'] for j in df[i].unique() ]
        print(i,len(a),len(df[i].unique()),end=' | ')
        col.append(i)
        pval.append(stats.f_oneway(*a).pvalue)
#pvalue = stats.f_oneway()

In [ ]:
anval = pd.DataFrame({'col':col,'pval':pval})
obj_col = list(anval.sort_values('pval')[0:65]['col'])
len(obj_col)

In [ ]:
df.shape,df1.shape

In [ ]:
c = df.corr()['SalePrice']
num_col = list(c[abs(c)>0.0].index)
len(num_col)

In [ ]:
col = num_col + obj_col

In [ ]:
df = df[col]
df1 = df1[[i for i in col if i!='SalePrice']]

In [ ]:
df.shape,df1.shape

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [ ]:
for i in obj_col:
    df[i]= label_encoder.fit_transform(df[i])
    df1[i]= label_encoder.fit_transform(df1[i])

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')

In [ ]:
df = df.astype(float)

In [ ]:
df = pd.DataFrame(imp.fit_transform(df),columns=list(df.columns))
df1 = pd.DataFrame(imp.fit_transform(df1),columns=list(df1.columns))

In [ ]:
# for i in df.select_dtypes(exclude='object'):
#     if df[i].nunique()>25:
#         df[i] = np.sqrt(df[i])
        
# for i in df1.select_dtypes(exclude='object'):
#     if df1[i].nunique()>25:
#         df1[i] = np.sqrt(df1[i])

In [ ]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [ ]:
X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sca = MinMaxScaler()

In [ ]:
X = pd.DataFrame(sca.fit_transform(X),columns=list(X.columns))
df1 = pd.DataFrame(sca.fit_transform(df1),columns=list(df1.columns))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model = CatBoostRegressor(iterations=10000,verbose=False)
#model = XGBRegressor(n_estimators=200)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
 #model = GridSearchCV(estimator=model1, param_grid = params, cv = 2, n_jobs=-1)

In [ ]:
model.fit(X_train,y_train)
model.score(X_train,y_train)

In [ ]:
# res = pd.DataFrame({'col':X.columns,'imp':model.feature_importances_})
# imp_col = list(res.sort_values('imp',ascending=False)[0:25]['col'])

In [ ]:
#res.sort_values('imp',ascending=False)[0:25]

In [ ]:
model.score(X_test,y_test)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_log_error
np.sqrt(mean_squared_log_error(y_test, y_pred))

In [ ]:
model.fit(X,y,verbose=False)
model.score(X,y)

In [ ]:
#model.get_all_params()

In [ ]:
y_pred = model.predict(df1)

In [ ]:
sub = pd.DataFrame({'id':range(1461,1461+len(df1)),'SalePrice':y_pred})
sub.to_csv('submission.csv',index=False)